In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Carrega a base de dados**

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

**Entendendo o dataset**

In [ ]:
print(boston.DESCR)

**Colunas da base de dados**

- CRIM: Taxa de criminalidade per capita por cidade.
- ZN: ProporÃ§Ã£o de terrenos residenciais divididos por lotes com mais de 25.000 pÃ©s quadrados.
- INDUS: ProporÃ§Ã£o de acres comerciais nÃ£o comerciais por cidade.
- CHAS: variÃ¡vel fictÃ­cia Charles River (= 1 se o trecho limita o rio; 0 caso contrÃ¡rio).
- NOX: concentraÃ§Ã£o de Ã³xido nÃ­trico (partes por 10 milhÃµes).
- RM: NÃºmero mÃ©dio de quartos por habitante.
- AGE: ProporÃ§Ã£o de unidades ocupadas pelos proprietÃ¡rios construÃ­das antes de 1940.
- DIS: DistÃ¢ncias ponderadas a cinco centros de emprego em Boston.
- RAD: Ãndice de acessibilidade Ã s rodovias radiais.
- TAX: taxa do imposto sobre a propriedade de valor total por US 10.000.
- PTRATIO: ProporÃ§Ã£o de alunos por professor por cidade.
- B: Bk Ã© a proporÃ§Ã£o de pessoas de descendÃªncia afro-americana por cidade.
- LSTAT: Porcentagem de status inferior da populaÃ§Ã£o.
- **MEDV**: Valor mÃ©dio das casas ocupadas pelos proprietÃ¡rios em US 1000.

**Separa os dados e as classes**

In [ ]:
boston.data

In [ ]:
boston.target

In [ ]:
dados = boston.data
classes = boston.target

In [ ]:
type(dados)

**Cria um dataframe pandas**

In [ ]:
df = pd.DataFrame(boston.data, columns = boston.feature_names)

In [ ]:
df.head()

In [ ]:
df.info()

**Checa missing values**

In [ ]:
df.isnull().sum()

### AnÃ¡lisando os dados ###

**Adiciona as classes ao dataset df para calcular a correlaÃ§Ã£o**

In [ ]:
df['classe'] = boston.target

**Plota a correlaÃ§Ã£o entre todas as colunas do dataframe**

In [ ]:
correlacoes = df.corr().round(2)
sns.heatmap(data=correlacoes, annot=False, linewidths=.5, cmap="YlGnBu")
plt.show()

**Pontos a observar**
- features com forte correlaÃ§Ã£o com a variÃ¡vel alvo
- features com multicorrelacionadas (RAD,TAX,INDUS)

**Visualizando Colunas Correlacionadas**

In [ ]:
import seaborn as sns 

In [ ]:
sns.set_style("darkgrid")
plt.scatter(df.RM, classes, color='red', marker='o')
plt.title('Numero medio de quartos por habitante vs Precos')
plt.xlabel('Numero medio de quartos por habitante')
plt.ylabel('Precos')
plt.show()

In [ ]:
sns.set_style("whitegrid")
plt.scatter(df.LSTAT, classes, color='green', marker='o')
plt.title('Porcentagem de status inferior da populacao vs Precos')
plt.xlabel('Porcentagem de status inferior da populacao')
plt.ylabel('Precos')
plt.show()

**Distribuicao dos precos**

In [ ]:
sns.distplot(classes, bins=50)
plt.title('Precos')
plt.show()

**Separa os dados em 20% para teste 80% treino**

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(dados, classes, test_size = 0.2)

In [ ]:
X_train

In [ ]:
Y_train

In [ ]:
X_test

In [ ]:
Y_test

** Importa as bibliotecas**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

**Treina a regressÃ£o linear**

In [ ]:
lin_model = LinearRegression()
lin_model.fit(X_train, Y_train)

**Testa o modelo**

In [ ]:
y_test_predicoes = lin_model.predict(X_test)

**Avaliando o modelo com mse e rmse**

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(Y_test, y_test_predicoes)
print('MSE: {}'.format(mse))

In [ ]:
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predicoes)))
print('RMSE: {}'.format(rmse))

## Construindo um Web Service
- Usado para colocar o modelo em produÃ§Ã£o.
- Permite integraÃ§Ã£o com aplicaÃ§Ãµes.

**Credenciais da conta**

In [ ]:
from azureml import Workspace
ws = Workspace()
workdspace_id = ws.workspace_id
authorization_token = ws.authorization_token

In [ ]:
workdspace_id

In [ ]:
authorization_token

**Construindo o Web Service e a funÃ§Ã£o ConsultaModelo()**

In [ ]:
from azureml import services
@services.publish(workdspace_id, authorization_token)
@services.types(crim=float, zn=float, indus=float, chas=float, nox=float, rm=float, age=float, 
                dis=float, rad=float, tax=float, ptratio=float, black=float, lstat=float)
@services.returns(float)
def ConsultaModelo(crim, zn, indus, chas, nox, rm, age, dis, rad, tax, ptratio, black, lstat):
    # predict the label
    feature_vector = [crim, zn, indus, chas, nox, rm, age, dis, rad, tax, ptratio, black, lstat]
    return lin_model.predict(feature_vector)

### Consumindo o Modelo
- Chama a funÃ§Ã£o ConsultaModelo() passando os dados de teste.

In [ ]:
# Features: crim, zn, indus, chas, nox, rm, age, dis, rad, tax, ptratio, black, lstat
ConsultaModelo(0.00632, 18, 3, 0, 0.538, 6.575, 65.2, 4.09, 1, 296, 15.3, 396.9, 4.98)

**Dados de credenciais**
- Usa os dados fora do Jupyter Notebook

In [ ]:
ConsultaModelo.service.url

In [ ]:
ConsultaModelo.service.api_key

In [ ]:
ConsultaModelo.service.help_url

In [ ]:
ConsultaModelo.service.service_id

**todas as variÃ¡veis juntas**

In [ ]:
service_url = ConsultaModelo.service.url 
api_key = ConsultaModelo.service.api_key
help_url = ConsultaModelo.service.help_url
service_id = ConsultaModelo.service.service_id

**Espera um tempo para construÃ§Ã£o do Web Service**

In [ ]:
import time
time.sleep(10)

** Use o cÃ³digo abaixo para consultar o web service**



[![Python Script][pic 1]][pic 1] Figure 1

[pic 1]: https://az712634.vo.msecnd.net/notebooks/d132b658-c5bd-11e5-97a3-0ba617781c98.PNG

In [ ]:
import urllib2
# If you are using Python 3+, import urllib instead of urllib2

import json 

data =  {

        "Inputs": {

                "input1":
                {
                    "ColumnNames": ["crim", "zn", "lstat", "age", "tax", "rad", "black", 
                                    "chas", "nox", "rm", "indus", "ptratio", "dis"],
                    "Values": [ [ "0.00632", "18", "4.98", "65.2", "296", "1", "396.9", 
                                 "0", "0.538", "6.575", "2.31", "15.3", "4.09" ], 
                               ["0.02731", "0", "9.14", "78.9", "242", "2", "396.9", 
                                "0", "0.469", "6.421", "7.07", "17.8", "4.9671"],
                            ]
                },        },
            "GlobalParameters": {
}
    }

body = str.encode(json.dumps(data))

url = service_url
api_key = api_key # Replace this with the API key for the web service

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib2.Request(url, body, headers) 

try:
    response = urllib2.urlopen(req)

    # If you are using Python 3+, replace urllib2 with urllib.request in the above code:
    # req = urllib.request.Request(url, body, headers) 
    # response = urllib.request.urlopen(req)

    result = response.read()
    print(result) 
except urllib2.HTTPError, error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, 
    # which are useful for debugging the failure
    print(error.info())

    print(json.loads(error.read()))                 